In [1]:
class Node:
    def __init__(self, data, depth=0, lchild=None, rchild=None):
        self.data = data
        self.depth = depth
        self.lchild = lchild
        self.rchild = rchild


class KdTree:
    def __init__(self):
        self.KdTree = None
        self.n = 0
        self.nearest = None

    def create(self, dataSet, depth=0):
        if len(dataSet) > 0:
            m, n = np.shape(dataSet)
            self.n = n - 1
            axis = depth % self.n
            mid = int(m / 2)
            dataSetcopy = sorted(dataSet, key=lambda x: x[axis])
            node = Node(dataSetcopy[mid], depth)
            if depth == 0:
                self.KdTree = node
            node.lchild = self.create(dataSetcopy[:mid], depth+1)
            node.rchild = self.create(dataSetcopy[mid+1:], depth+1)
            return node
        return None

    def preOrder(self, node):
        if node is not None:
            print(node.depth, node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)

    def search(self, x, count=1):
        nearest = []
        for i in range(count):
            nearest.append([-1, None])
        self.nearest = np.array(nearest)

        def recurve(node):
            if node is not None:
                axis = node.depth % self.n
                daxis = x[axis] - node.data[axis]
                if daxis < 0:
                    recurve(node.lchild)
                else:
                    recurve(node.rchild)

                dist = sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(x, node.data)))
                for i, d in enumerate(self.nearest):
                    if d[0] < 0 or dist < d[0]:
                        self.nearest = np.insert(self.nearest, i, [dist, node], axis=0)
                        self.nearest = self.nearest[:-1]
                        break

                n = list(self.nearest[:, 0]).count(-1)
                if self.nearest[-n-1, 0] > abs(daxis):
                    if daxis < 0:
                        recurve(node.rchild)
                    else:
                        recurve(node.lchild)

        recurve(self.KdTree)

        knn = self.nearest[:, 1]
        belong = []
        for i in knn:
            belong.append(i.data[-1])
        b = max(set(belong), key=belong.count)

        return self.nearest, b


In [2]:
%%time
import numpy as np
from math import sqrt
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn import preprocessing
min_max_scaler=preprocessing.MinMaxScaler()

def run(df):  
    dist2=[]
    data2=np.array(df.drop('label',axis=1))
    for m in range(0,len(data2)):
        for n in range(m+1,len(data2)):
             dist2.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a = np.percentile(dist2,10) #密度阈值(%10分位数)
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(7 / 3)/ 7])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值

    F_list,Re_list=[],[]
    for rate in[0.05,0.1,0.2,0.3,0.4]:
        P,R,Fh,Re=[],[],[],[]
        for term in range(10): #多次试验取均值
            
            noise_set=pd.DataFrame(columns=df.columns)
            for r in range(0,(df.label.max()+1)):    
                noise_set=pd.concat([noise_set,df[df.label==r].sample(frac=rate, replace=False)]) #选取噪声比例  
            #bb=df[df.label==1].sample(frac=rate, replace=False) #选取噪声比例
            #cc=df[df.label==2].sample(frac=rate, replace=False) #选取噪声比例

            data_cut=df[~df.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(int(np.random.rand(1)[0]*(df.label.max()+1))%10)
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    noise_set.loc[l,'label2']=(noise_set.loc[l,'label']+1) % (df.label.max()+1)
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            data= pd.concat([data_cut,noise],axis=0,ignore_index=True) #含噪声数据集

            data=np.array(data)
            kdt = KdTree()
            kdt.create(data)
            # kdt.preOrder(kdt.KdTree)
            
            print('epoch..........:',term)
            Dn = []
            for x in data:
                #plt.scatter(x[0], x[1], c='red', marker='x')  # 测试点
                near, belong = kdt.search(x[:-1], 8)  # 设置临近点的个数
                #print("test:")
               # print(x, "predict:", belong)
               # print("nearest:")
                density = 0
                t = 0
                for n in near:
                    #print(n[1].data, "dist:", n[0])
                    #plt.scatter(n[1].data[0], n[1].data[1], c='green', marker='+')  # k个最近邻点
                    density += n[0]  #密度
                    if (x[-1] == n[1].data[-1])and (x != n[1].data).any():
                        t += 1
                Ps = t / 7
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)

               # print(densit)
                if (density <= a): #高密度区域
                    if x[-1] != belong:
                        Dn.append(list(x))

                        #print('高密度区域噪声')
                else:
                    if (H < H_): #低密度单一标签区域
                        if x[-1] != belong:
                            Dn.append(list(x))
                           # print('H:',H)
                           # print('低密度单一区域噪声')
                    else: #低密度混合标签区域
                        hon,hen= 0,0
                        dist_hon,dist_hen = 0,0
                        for n in near:
                            if x[-1]==n[1].data[-1]and(x != n[1].data).any():
                                hon += 1
                                dist_hon += n[0]
                            elif x[-1] != n[1].data[-1]:
                                hen +=1
                                dist_hen += n[0]
                        if (dist_hon / hon) / (dist_hen / hen) > 1.2:
                            Dn.append(list(x))
                           # print('H:',H)
                           # print('低密度混合标签区域噪声')

            #print('标签噪声：',Dn)
            noise_list=noise.values.tolist()
            TP = 0
            for c in Dn:
                if c in noise_list:
                    TP += 1
            FP = len(Dn) - TP
            FN = len(noise_list) - TP
            precision= TP / (TP + FP)
            recall= TP / (TP + FN)
            fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
        P.append(precision) #准确率
        R.append(recall)#召回率
        Fh.append(fh) # F0.5
        Re.append(len(Dn) / len(data)) #移除率
        print(rate*100,'%','噪声下:')
        print('准确率:',np.mean(P))
        print('召回率:',np.mean(R))
        print('F0.5值:',np.mean(Fh))
        print('移除率:',np.mean(Re))
        F_list.append(np.mean(Fh))
        Re_list.append(np.mean(Re))
    return F_list,Re_list

Wall time: 3.13 s


In [3]:
%%time
letter=pd.read_csv('./mydata/letter.csv')
letter_sample=letter.drop(['label'],axis=1)
letter_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(letter_sample)))
letter_sample['label']=letter.label
letter_sample=letter_sample.drop_duplicates()
result=run(letter_sample)

epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5
epoch..........: 6
epoch..........: 7
epoch..........: 8
epoch..........: 9
5.0 % 噪声下:
准确率: 0.9819819819819819
召回率: 0.990909090909091
F0.5值: 0.9837545126353789
移除率: 0.050546448087431695
epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5
epoch..........: 6
epoch..........: 7
epoch..........: 8
epoch..........: 9
10.0 % 噪声下:
准确率: 0.9734513274336283
召回率: 1.0
F0.5值: 0.9786476868327402
移除率: 0.1029143897996357
epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5
epoch..........: 6
epoch..........: 7
epoch..........: 8
epoch..........: 9
20.0 % 噪声下:
准确率: 0.9432314410480349
召回率: 0.9840546697038725
F0.5值: 0.9511228533685601
移除率: 0.20856102003642987
epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5


#原方法

In [4]:
%%time
def run2(df):  
    dist2=[]
    data2=np.array(df.drop('label',axis=1))
    for m in range(0,len(data2)):
        for n in range(m+1,len(data2)):
             dist2.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a = np.percentile(dist2,10) #密度阈值(%10分位数)
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(7 / 3)/ 7])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值

    F_list,Re_list=[],[]
    for rate in[0.05,0.1,0.2,0.3,0.4]:
        P,R,Fh,Re=[],[],[],[]
        for term in range(10):#多次试验取均值

            noise_set=pd.DataFrame(columns=df.columns)
            for r in range(0,(df.label.max()+1)):    
                noise_set=pd.concat([noise_set,df[df.label==r].sample(frac=rate, replace=False)]) #选取噪声比例  
            #bb=df[df.label==1].sample(frac=rate, replace=False) #选取噪声比例
            #cc=df[df.label==2].sample(frac=rate, replace=False) #选取噪声比例

            data_cut=df[~df.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(int(np.random.rand(1)[0]*(df.label.max()+1))%10)
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    noise_set.loc[l,'label2']=(noise_set.loc[l,'label']+1) % (df.label.max()+1)
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            data= pd.concat([data_cut,noise],axis=0,ignore_index=True) #含噪声数据集

            data=np.array(data)
            kdt = KdTree()
            kdt.create(data)
            # kdt.preOrder(kdt.KdTree)

            print('epoch..........:',term)
            Dn = []
            for x in data:
                #plt.scatter(x[0], x[1], c='red', marker='x')  # 测试点
                near, belong = kdt.search(x[:-1], 8)  # 设置临近点的个数
                #print("test:")
               # print(x, "predict:", belong)
               # print("nearest:")
                density = 0
                t,hon,hen,dist_hon,dist_hen,DoD,DRL,count = 0,0,0,0,0,0,0,0
                for n in near:
                    if x[-1] == n[1].data[-1] and (x != n[1].data).any():
                        t += 1
                        hon += 1
                        dist_hon += n[0]

                    elif  x[-1] != n[1].data[-1]:
                        hen += 1
                        dist_hen += n[0]

                    density += n[0]  #密度   

                DoD = abs(dist_hen - dist_hon )#相异性差值(加绝对值)
                DRL = hen - hon  #标签异同差比    
                Ps = t / 7
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
               # print(densit)
                if (density <= a): #高密度区域
                    if DRL > 0:
                        Dn.append(list(x))

                else:
                    if (H < H_): #低密度单一标签区域
                        if DRL > 0:
                            Dn.append(list(x))

                    else: #低密度混合标签区   
                        if DoD >= (density / 7):
                            Sa,hon_sa,hen_sa,DoD_sa=0,0,0,0
                            near2=[]
                            for n in near:
                                Sa += n[0]
                                if Sa <= a:
                                    near2.append(n)
                            for n2 in near2:
                                if (x[-1]==n2[1].data[-1])and (x != n2[1].data).any():
                                    hon_sa +=1
                                elif x[-1] !=n2[1].data[-1]:
                                    hen_sa +=1
                                DoD_sa = hen_sa - hon_sa
                            if DoD_sa > 0:        
                                Dn.append(list(x))

                            elif DoD_sa == 0 and x[-1] != near[len(near2)][1].data[-1]:
                                Dn.append(list(x))  

            #print('标签噪声：',Dn)
            noise_list=noise.values.tolist()
            TP = 0
            for c in Dn:
                if c in noise_list:
                    TP += 1
            FP = len(Dn) - TP
            FN = len(noise_list) - TP
            precision= TP / (TP + FP)
            recall= TP / (TP + FN)
            fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
        P.append(precision) #准确率
        R.append(recall)#召回率
        Fh.append(fh) # F0.5
        Re.append(len(Dn) / len(data)) #移除率
        print(rate*100,'%','噪声下:')
        print('准确率:',np.mean(P))
        print('召回率:',np.mean(R))
        print('F0.5值:',np.mean(Fh))
        print('移除率:',np.mean(Re))
        F_list.append(np.mean(Fh))
        Re_list.append(np.mean(Re))
    return F_list,Re_list

Wall time: 0 ns


In [5]:
result2=run2(letter_sample)

epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5
epoch..........: 6
epoch..........: 7
epoch..........: 8
epoch..........: 9
5.0 % 噪声下:
准确率: 0.9734513274336283
召回率: 1.0
F0.5值: 0.9786476868327402
移除率: 0.05145719489981785
epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5
epoch..........: 6
epoch..........: 7
epoch..........: 8
epoch..........: 9
10.0 % 噪声下:
准确率: 0.9361702127659575
召回率: 1.0
F0.5值: 0.9482758620689655
移除率: 0.10701275045537341
epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5
epoch..........: 6
epoch..........: 7
epoch..........: 8
epoch..........: 9
20.0 % 噪声下:
准确率: 0.8304761904761905
召回率: 0.9931662870159453
F0.5值: 0.8586057502953918
移除率: 0.2390710382513661
epoch..........: 0
epoch..........: 1
epoch..........: 2
epoch..........: 3
epoch..........: 4
epoch..........: 5
epoch..........

In [6]:
result2

([0.9786476868327402,
  0.9482758620689655,
  0.8586057502953918,
  0.7927606423655364,
  0.7443033246171088],
 [0.05145719489981785,
  0.10701275045537341,
  0.2390710382513661,
  0.37158469945355194,
  0.5095628415300546])

In [7]:
result

([0.9837545126353789,
  0.9786476868327402,
  0.9511228533685601,
  0.9009141846063107,
  0.8630013519603424],
 [0.050546448087431695,
  0.1029143897996357,
  0.20856102003642987,
  0.31102003642987247,
  0.40528233151183973])

In [8]:
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib

Using matplotlib backend: Qt5Agg


In [9]:
noise_rate=[5,10,20,30,40]
fig = plt.figure()
ax1 = axislines.Subplot(fig, 1,1,1)
fig.add_subplot(ax1)
ax1.set_xticks(noise_rate)
ax1.axis([0,40,0,1])
plt.plot(noise_rate,result[0],'ro-',label='IM_DDF')
plt.plot(noise_rate,result2[0],'b*-.',label='DDF')
plt.xlabel('噪声比例/%')
plt.ylabel('F0.5')
plt.title('不同噪声下的F0.5值',fontsize=16)
plt.legend()
plt.show()

In [10]:
fig2 = plt.figure()
ax2 = axislines.Subplot(fig2, 1,1,1)
fig2.add_subplot(ax2)
ax2.set_xticks([5,10,20,30,40])
ax2.set_yticks([0,0.2,0.4,0.6,0.8])
ax2.axis([0,40,0,0.8])
plt.plot(noise_rate,result[1],'ro-',label='IM_DDF')
plt.plot(noise_rate,result2[1],'b*-.',label='DDF')
plt.xlabel('噪声比例/%')
plt.ylabel('Re')
plt.title('不同噪声下的过滤值Re',fontsize=16)
plt.legend()
plt.show()